## Install necessary dependencies

In [1]:
!pip install transformers
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 17.6 MB/s 
     |████████████████████████████████| 182 kB 62.5 MB/s 
     |████████████████████████████████| 7.6 MB 59.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 6.5 MB/s 


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import pandas as pd

## Load the model and tokenizer

In [4]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

## Define functions

In [5]:
## define the function to download the hashtag related tweets

def tweet_hashtag_scrapper(query, n_tweet):
  attributes_container = []
  max_tweet = n_tweet

  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):

    if i>max_tweet:
        break
        
    attributes_container.append([tweet.date,tweet.user.username,tweet.user.followersCount,tweet.likeCount,tweet.content,tweet.hashtags])
    
  return pd.DataFrame(attributes_container, columns=["Date","User","FollowersCount","LikeCount","Content","Hashtags"])

In [6]:
## 推文情绪分析函数
def twitter_sentiment(tweet):
  tweet_words = []
  for word in tweet.split(' '):
      if word.startswith('@') and len(word) > 1:
          word = '@user'
      elif word.startswith('http'):
          word = "http"
      tweet_words.append(word)
  tweet_proc = " ".join(tweet_words)
  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  def sentiment_evalue (scores):
    max_score = np.argmax(scores)
    if max_score == 0:
      return "Negative"
    elif max_score == 1:
      return "Neutral"
    else:
      return "Positive"
  return sentiment_evalue(scores)

## Download tweets for a specific user and calculate the weekly growth rate

#### Moonbeam

In [7]:
Moonbeam_query = "(from:MoonbeamNetwork) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(Moonbeam_query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_moonbeam = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])


<ipython-input-7-13c259005242>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [8]:
df_moonbeam.head(15)

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-23 15:46:45+00:00,MoonbeamNetwork,📢 Calling our Chinese-speaking community! \n\n...,280126,76,https://twitter.com/MoonbeamNetwork/status/160...
1,2022-12-23 12:14:56+00:00,MoonbeamNetwork,@Fiskantes Seeing builders leverage #connected...,280126,7,https://twitter.com/MoonbeamNetwork/status/160...
2,2022-12-23 11:15:57+00:00,MoonbeamNetwork,GM everyone🎄 https://t.co/ATkNGCPqTQ,280126,220,https://twitter.com/MoonbeamNetwork/status/160...
3,2022-12-22 23:31:12+00:00,MoonbeamNetwork,2/ The public endpoint service is enabled by 8...,280126,30,https://twitter.com/MoonbeamNetwork/status/160...
4,2022-12-22 23:31:12+00:00,MoonbeamNetwork,1/ A collective of collators on Moonbeam &amp;...,280126,110,https://twitter.com/MoonbeamNetwork/status/160...
5,2022-12-22 19:46:59+00:00,MoonbeamNetwork,⛓️🔄⛓️ Learn more about XCM on Polkadot &amp; h...,280126,60,https://twitter.com/MoonbeamNetwork/status/160...
6,2022-12-22 19:46:59+00:00,MoonbeamNetwork,5. Priming for new capabilities\n\na. XCM is v...,280126,45,https://twitter.com/MoonbeamNetwork/status/160...
7,2022-12-22 19:46:59+00:00,MoonbeamNetwork,4. Simplifying transactions\n\na. XCM simplifi...,280126,34,https://twitter.com/MoonbeamNetwork/status/160...
8,2022-12-22 19:46:59+00:00,MoonbeamNetwork,3. Improving UX\n\na. XCM enables Moonbeam dAp...,280126,37,https://twitter.com/MoonbeamNetwork/status/160...
9,2022-12-22 19:46:58+00:00,MoonbeamNetwork,2. Resolving compatibility barriers\n\na. XCMP...,280126,41,https://twitter.com/MoonbeamNetwork/status/160...


In [9]:
# Calculate Followers weekly growth rate
# Extract the date 
df_moonbeam['only_date'] = df_moonbeam['Date'].dt.date
df_moonbeam.head()


,Date,User,Tweet,FollowersCount,likeCount,url,only_date
0,2022-12-23 15:46:45+00:00,MoonbeamNetwork,📢 Calling our Chinese-speaking community! \n\n...,280126,76,https://twitter.com/MoonbeamNetwork/status/160...,2022-12-23
1,2022-12-23 12:14:56+00:00,MoonbeamNetwork,@Fiskantes Seeing builders leverage #connected...,280126,7,https://twitter.com/MoonbeamNetwork/status/160...,2022-12-23
2,2022-12-23 11:15:57+00:00,MoonbeamNetwork,GM everyone🎄 https://t.co/ATkNGCPqTQ,280126,220,https://twitter.com/MoonbeamNetwork/status/160...,2022-12-23
3,2022-12-22 23:31:12+00:00,MoonbeamNetwork,2/ The public endpoint service is enabled by 8...,280126,30,https://twitter.com/MoonbeamNetwork/status/160...,2022-12-22
4,2022-12-22 23:31:12+00:00,MoonbeamNetwork,1/ A collective of collators on Moonbeam &amp;...,280126,110,https://twitter.com/MoonbeamNetwork/status/160...,2022-12-22


In [27]:
#convert the 'Date' column to datetime objects
df_moonbeam['only_date'] = pd.to_datetime(df_moonbeam['only_date'])

# group the data by week
df_moonbeam_weekly = df_moonbeam.groupby(df_moonbeam['only_date'].dt.week).sum()  ## compare the user growth rate by the total number of week


# # Next, filter the data to include only Mondays
# df_moonbeam_monday = df_moonbeam[df_moonbeam['only_date'].dt.dayofweek == 0]

# Calculate the weekly user growth rate
df_moonbeam_weekly['growth_rate'] = df_moonbeam_weekly['FollowersCount'].pct_change()

df_moonbeam_weekly.head(10)

<ipython-input-27-204e8de592fd>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_moonbeam_weekly = df_moonbeam.groupby(df_moonbeam['only_date'].dt.week).sum()  ## compare the user growth rate by the total number of week


,FollowersCount,likeCount,growth_rate
only_date,,,
47,2801260,847,NaN
48,22410080,5928,7.000000
49,17928064,4329,-0.200000
50,15126804,3855,-0.156250
51,13446048,2885,-0.111111


#### Acala

In [19]:
Moonbeam_query = "(from:AcalaNetwork) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(Moonbeam_query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Acala = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])


<ipython-input-19-98a9e6b31a2d>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [20]:
df_Acala.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-22 13:45:39+00:00,AcalaNetwork,A vote for @gauntletnetwork’s latest risk para...,265731,80,https://twitter.com/AcalaNetwork/status/160592...
1,2022-12-19 16:01:59+00:00,AcalaNetwork,To begin building a squid on Acala or @KaruraN...,265731,30,https://twitter.com/AcalaNetwork/status/160486...
2,2022-12-19 16:01:58+00:00,AcalaNetwork,We are excited to release support from @subsqu...,265731,130,https://twitter.com/AcalaNetwork/status/160486...
3,2022-12-19 13:45:06+00:00,AcalaNetwork,You also find a video version of the 2023 road...,265731,28,https://twitter.com/AcalaNetwork/status/160483...
4,2022-12-19 13:45:06+00:00,AcalaNetwork,"Last but not least, Governance will be another...",265731,43,https://twitter.com/AcalaNetwork/status/160483...


In [29]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Acala['only_date'] = df_Acala['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Acala['only_date'] = pd.to_datetime(df_Acala['only_date'])
# group the data by week
df_Acala_weekly = df_Acala.groupby(df_Acala['only_date'].dt.week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Acala_weekly['growth_rate'] = df_Acala_weekly['FollowersCount'].pct_change()

df_Acala_weekly.head(10)

<ipython-input-29-7d73b951afef>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_Acala_weekly = df_Acala.groupby(df_Acala['only_date'].dt.week).sum()  ## compare the user growth rate by the total number of week


,FollowersCount,likeCount,growth_rate
only_date,,,
48,4783158,1316,NaN
49,6111813,1829,0.277778
50,4251696,1144,-0.304348
51,3720234,695,-0.125000


#### Astar

In [32]:
query = "(from:AstarNetwork) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Astar = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-32-08c1bddbc7cf>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [33]:
df_Astar.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-23 21:26:33+00:00,AstarNetwork,https://t.co/c520SFxOB5,349046,24,https://twitter.com/AstarNetwork/status/160640...
1,2022-12-23 21:26:33+00:00,AstarNetwork,https://t.co/1S0TywD4e8,349046,34,https://twitter.com/AstarNetwork/status/160640...
2,2022-12-23 21:26:32+00:00,AstarNetwork,https://t.co/dVYPf82u2Y,349046,27,https://twitter.com/AstarNetwork/status/160640...
3,2022-12-23 21:26:31+00:00,AstarNetwork,https://t.co/mtl6KOS3zd,349046,16,https://twitter.com/AstarNetwork/status/160640...
4,2022-12-23 21:26:31+00:00,AstarNetwork,https://t.co/f1Ekw7t76R,349046,30,https://twitter.com/AstarNetwork/status/160640...


In [34]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Astar['only_date'] = df_Astar['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Astar['only_date'] = pd.to_datetime(df_Astar['only_date'])
# group the data by week
df_Astar_weekly = df_Astar.groupby(df_Astar['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Astar_weekly['growth_rate'] = df_Astar_weekly['FollowersCount'].pct_change()

df_Astar_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,10122334,2002,NaN
48,13961840,2960,0.379310
49,18848484,3755,0.350000
50,16056116,2874,-0.148148
51,17452300,3626,0.086957


#### Centrifuge

In [35]:
query = "(from:centrifuge) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Centrifuge = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-35-50f4a79fecc7>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [36]:
df_Centrifuge.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-23 16:56:54+00:00,centrifuge,4/ And in case you missed our announcement: he...,58641,23,https://twitter.com/centrifuge/status/16063330...
1,2022-12-23 16:56:53+00:00,centrifuge,3/ Linked below are the assets — financed on-c...,58641,18,https://twitter.com/centrifuge/status/16063330...
2,2022-12-23 16:56:53+00:00,centrifuge,2/ These assets are part of the BlockTower Ser...,58641,24,https://twitter.com/centrifuge/status/16063330...
3,2022-12-23 16:56:52+00:00,centrifuge,"1/ Last week, we announced that $220 million i...",58641,210,https://twitter.com/centrifuge/status/16063330...
4,2022-12-23 00:15:21+00:00,centrifuge,More updates in this department coming soon 🌀 ...,58641,48,https://twitter.com/centrifuge/status/16060809...


In [37]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Centrifuge['only_date'] = df_Centrifuge['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Centrifuge['only_date'] = pd.to_datetime(df_Centrifuge['only_date'])
# group the data by week
df_Centrifuge_weekly = df_Centrifuge.groupby(df_Centrifuge['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Centrifuge_weekly['growth_rate'] = df_Centrifuge_weekly['FollowersCount'].pct_change()

df_Centrifuge_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
48,586410,239,NaN
49,527769,314,-0.100000
50,645051,578,0.222222
51,293205,323,-0.545455


#### Phala

In [39]:
query = "(from:PhalaNetwork) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Phala = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-39-757b191b0221>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [40]:
df_Phala.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-22 15:28:21+00:00,PhalaNetwork,📚Here are some material to help you understand...,118010,30,https://twitter.com/PhalaNetwork/status/160594...
1,2022-12-22 15:28:18+00:00,PhalaNetwork,Phala App v2.0 has arrived! 🎉🎉\n\nStart your n...,118010,109,https://twitter.com/PhalaNetwork/status/160594...
2,2022-12-22 13:42:22+00:00,PhalaNetwork,@Rod26706215 Yes. It was posted in our Discord...,118010,0,https://twitter.com/PhalaNetwork/status/160592...
3,2022-12-22 13:40:28+00:00,PhalaNetwork,@PapsIverson https://t.co/FdnvNwKrGN,118010,1,https://twitter.com/PhalaNetwork/status/160592...
4,2022-12-22 13:31:38+00:00,PhalaNetwork,@2km_sola Please use the following form https:...,118010,0,https://twitter.com/PhalaNetwork/status/160591...


In [41]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Phala['only_date'] = df_Phala['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Phala['only_date'] = pd.to_datetime(df_Phala['only_date'])
# group the data by week
df_Phala_weekly = df_Phala.groupby(df_Phala['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Phala_weekly['growth_rate'] = df_Phala_weekly['FollowersCount'].pct_change()

df_Phala_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,472040,279,NaN
48,3422290,1453,6.250000
49,944080,564,-0.724138
50,2360200,880,1.500000
51,1770150,572,-0.250000


#### Efinity

In [42]:
query = "(from:efinityio) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Efinity = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-42-8ae2144c69fd>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [43]:
df_Efinity.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-20 12:03:28+00:00,efinityio,It’s time for the last #Efinity development up...,70922,72,https://twitter.com/efinityio/status/160517202...
1,2022-12-12 12:13:09+00:00,efinityio,"First ever #Efinity Beam NFT is here, grab you...",70922,95,https://twitter.com/efinityio/status/160227536...
2,2022-12-09 11:00:02+00:00,efinityio,The team over at @enjin have brought out their...,70922,73,https://twitter.com/efinityio/status/160116979...
3,2022-12-06 12:00:00+00:00,efinityio,Another dev update is upon us! \n\nWe’ve made ...,70922,127,https://twitter.com/efinityio/status/160009772...
4,2022-11-24 17:00:00+00:00,efinityio,Happy Thanksgiving from the Efinity team! 🦃\n\...,70922,131,https://twitter.com/efinityio/status/159582456...


In [44]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Efinity['only_date'] = df_Efinity['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Efinity['only_date'] = pd.to_datetime(df_Efinity['only_date'])
# group the data by week
df_Efinity_weekly = df_Efinity.groupby(df_Efinity['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Efinity_weekly['growth_rate'] = df_Efinity_weekly['FollowersCount'].pct_change()

df_Efinity_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,70922,131,NaN
49,141844,200,1.0
50,70922,95,-0.5
51,70922,72,0.0


#### KILT

In [45]:
query = "(from:Kiltprotocol) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_KILT = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-45-865a88612c53>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [46]:
df_KILT.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-20 13:32:52+00:00,Kiltprotocol,"Watch @ingoruebe explain how the web evolved, ...",71400,79,https://twitter.com/Kiltprotocol/status/160519...
1,2022-12-15 10:11:21+00:00,Kiltprotocol,There’s a new proposal to implement a Constitu...,71400,79,https://twitter.com/Kiltprotocol/status/160333...
2,2022-12-09 12:07:51+00:00,Kiltprotocol,“There are a lot of very committed teams build...,71400,41,https://twitter.com/Kiltprotocol/status/160118...
3,2022-12-09 12:07:51+00:00,Kiltprotocol,“We have to protect the @Polkadot name and mak...,71400,144,https://twitter.com/Kiltprotocol/status/160118...
4,2022-12-07 13:39:39+00:00,Kiltprotocol,"Join tomorrows ""Meet the Web3 Builders"" @Polka...",71400,77,https://twitter.com/Kiltprotocol/status/160048...


In [47]:
# Calculate Followers weekly growth rate
# Extract the date 
df_KILT['only_date'] = df_KILT['Date'].dt.date
#convert the 'Date' column to datetime objects
df_KILT['only_date'] = pd.to_datetime(df_KILT['only_date'])
# group the data by week
df_KILT_weekly = df_KILT.groupby(df_KILT['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_KILT_weekly['growth_rate'] = df_KILT_weekly['FollowersCount'].pct_change()

df_KILT_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,71400,178,NaN
48,1570800,1477,21.000000
49,285600,355,-0.818182
50,71400,79,-0.750000
51,71400,79,0.000000


#### Litentry

In [48]:
query = "(from:litentry) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_litentry = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-48-e3de5823d054>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [49]:
df_litentry.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-23 11:28:25+00:00,litentry,A Merry Web3 Christmas to the Community! May y...,70188,33,https://twitter.com/litentry/status/1606250371...
1,2022-12-22 08:05:51+00:00,litentry,We’re also giving away 200$ in $LIT for the 5 ...,70188,17,https://twitter.com/litentry/status/1605837005...
2,2022-12-22 07:57:19+00:00,litentry,Catch two of our team members during this spec...,70188,27,https://twitter.com/litentry/status/1605834858...
3,2022-12-20 04:47:13+00:00,litentry,"'Tis the season!🎄Our cultures, communities, an...",70188,28,https://twitter.com/litentry/status/1605062239...
4,2022-12-19 09:42:36+00:00,litentry,@twobitidiot Litentry is building something ex...,70188,30,https://twitter.com/litentry/status/1604774189...


In [50]:
# Calculate Followers weekly growth rate
# Extract the date 
df_litentry['only_date'] = df_litentry['Date'].dt.date
#convert the 'Date' column to datetime objects
df_litentry['only_date'] = pd.to_datetime(df_litentry['only_date'])
# group the data by week
df_litentry_weekly = df_litentry.groupby(df_litentry['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_litentry_weekly['growth_rate'] = df_litentry_weekly['FollowersCount'].pct_change()

df_litentry_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
48,912444,341,NaN
49,421128,396,-0.538462
50,561504,253,0.333333
51,350940,135,-0.375000


#### Darwinia

In [51]:
query = "(from:DarwiniaNetwork) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Darwinia = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-51-3c427874d995>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [52]:
df_Darwinia.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-19 14:05:44+00:00,DarwiniaNetwork,@MangataFinance @oak_network @ImbueNetwork @bi...,70154,19,https://twitter.com/DarwiniaNetwork/status/160...
1,2022-12-19 14:05:21+00:00,DarwiniaNetwork,@MangataFinance @oak_network @ImbueNetwork @bi...,70154,19,https://twitter.com/DarwiniaNetwork/status/160...
2,2022-12-19 14:04:56+00:00,DarwiniaNetwork,"#PolkaPals Christmas 🎄 Giveaway.\nWin 10,282 $...",70154,221,https://twitter.com/DarwiniaNetwork/status/160...
3,2022-12-15 08:56:58+00:00,DarwiniaNetwork,"🚩 Crab has taken Para ID: 2229 lease period, a...",70154,35,https://twitter.com/DarwiniaNetwork/status/160...
4,2022-12-13 03:17:41+00:00,DarwiniaNetwork,💬 Discussion\nParticipate in discussions and l...,70154,8,https://twitter.com/DarwiniaNetwork/status/160...


In [53]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Darwinia['only_date'] = df_Darwinia['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Darwinia['only_date'] = pd.to_datetime(df_Darwinia['only_date'])
# group the data by week
df_Darwinia_weekly = df_Darwinia.groupby(df_Darwinia['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Darwinia_weekly['growth_rate'] = df_Darwinia_weekly['FollowersCount'].pct_change()

df_Darwinia_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,140308,58,NaN
48,420924,76,2.000000
49,210462,69,-0.500000
50,350770,81,0.666667
51,210462,259,-0.400000


#### Acurast

In [54]:
query = "(from:Acurast) until:2022-12-24 since:2022-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Acurast = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-54-0234ce6c53a8>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [55]:
df_Acurast.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-23 14:00:35+00:00,Acurast,"Data Transmitters, Data Consumers, and a Seaml...",408,16,https://twitter.com/Acurast/status/16062886617...
1,2022-12-23 13:00:54+00:00,Acurast,Celebrate the holidays with exclusive Acurast ...,408,17,https://twitter.com/Acurast/status/16062736453...
2,2022-12-23 07:54:34+00:00,Acurast,Congrats to our friends at @pendulum_chain for...,408,14,https://twitter.com/Acurast/status/16061965514...
3,2022-12-22 17:31:12+00:00,Acurast,@KryptosChain https://t.co/z2AZ8JD3KB,408,1,https://twitter.com/Acurast/status/16059792799...
4,2022-12-22 16:23:18+00:00,Acurast,Make that two! 🙌\n\nWe are excited that @pend...,408,18,https://twitter.com/Acurast/status/16059621925...


In [56]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Acurast['only_date'] = df_Acurast['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Acurast['only_date'] = pd.to_datetime(df_Acurast['only_date'])
# group the data by week
df_Acurast_weekly = df_Acurast.groupby(df_Acurast['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Acurast_weekly['growth_rate'] = df_Acurast_weekly['FollowersCount'].pct_change()

df_Acurast_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
47,1632,53,NaN
48,3264,152,1.000000
49,4488,106,0.375000
50,6528,132,0.454545
51,8568,166,0.312500


#### BSY

In [62]:
query = "(from:Bestay_bsy) until:2022-12-24 since:2021-07-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Bsy = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

In [63]:
df_Bsy.head()

,Date,User,Tweet,FollowersCount,likeCount,url




#### Diora



In [69]:
query = "(from:DioraNetwork) until:2022-12-24 since:2021-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Diora = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-69-a0bde4d1dfab>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [67]:
df_Diora.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-06-02 11:35:53+00:00,DioraNetwork,We have published a new article explaining the...,345,5,https://twitter.com/DioraNetwork/status/153232...
1,2022-05-25 20:27:49+00:00,DioraNetwork,Our landing Page is live 🎉\n\n👉 https://t.co/T...,345,7,https://twitter.com/DioraNetwork/status/152955...
2,2022-05-17 11:22:03+00:00,DioraNetwork,🚨 Announcing Diora Network 🚨\n\nCompeting in t...,345,16,https://twitter.com/DioraNetwork/status/152652...


In [68]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Diora['only_date'] = df_Diora['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Diora['only_date'] = pd.to_datetime(df_Diora['only_date'])
# group the data by week
df_Diora_weekly = df_Diora.groupby(df_Diora['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Diora_weekly['growth_rate'] = df_Diora_weekly['FollowersCount'].pct_change()

df_Diora_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
20,345,16,NaN
21,345,7,0.0
22,345,5,0.0


#### ShiftNrg

In [70]:
query = "(from:ShiftNrg) until:2022-12-24 since:2021-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_ShiftNrg = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

In [72]:
df_ShiftNrg.head()

,Date,User,Tweet,FollowersCount,likeCount,url


#### Kylin

In [73]:
query = "(from:Kylin_Network) until:2022-12-24 since:2021-11-24"  # Twitter account and date range
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])
        
df_Kylin = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'FollowersCount', 'likeCount','url'])

<ipython-input-73-47c1fe9dd5b7>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount,tweet.url])


In [74]:
df_Kylin.head()

,Date,User,Tweet,FollowersCount,likeCount,url
0,2022-12-22 20:24:36+00:00,Kylin_Network,Kylin Network Achievement in 2022 🚀\n\nHappy H...,70269,41,https://twitter.com/Kylin_Network/status/16060...
1,2022-12-22 01:07:00+00:00,Kylin_Network,Check #Kylin out on the latest version of @sub...,70269,23,https://twitter.com/Kylin_Network/status/16057...
2,2022-12-19 18:57:41+00:00,Kylin_Network,Kylin Monthly Technical Update: Nov 2022\n\n▪️...,70269,47,https://twitter.com/Kylin_Network/status/16049...
3,2022-12-12 17:29:21+00:00,Kylin_Network,A Message to our Community.\n\nhttps://t.co/lG...,70269,49,https://twitter.com/Kylin_Network/status/16023...
4,2022-12-09 11:21:42+00:00,Kylin_Network,Take responsibility for your data; it’s worth ...,70269,69,https://twitter.com/Kylin_Network/status/16011...


In [75]:
# Calculate Followers weekly growth rate
# Extract the date 
df_Kylin['only_date'] = df_Kylin['Date'].dt.date
#convert the 'Date' column to datetime objects
df_Kylin['only_date'] = pd.to_datetime(df_Kylin['only_date'])
# group the data by week
df_Kylin_weekly = df_Kylin.groupby(df_Kylin['only_date'].dt.isocalendar().week).sum()  ## compare the user growth rate by the total number of week

# Calculate the weekly user growth rate
df_Kylin_weekly['growth_rate'] = df_Kylin_weekly['FollowersCount'].pct_change()

df_Kylin_weekly.head(10)

,FollowersCount,likeCount,growth_rate
week,,,
1,1686456,2268,NaN
2,562152,1019,-0.666667
3,562152,1030,0.000000
4,702690,1076,0.250000
5,1054035,1351,0.500000
6,1054035,1505,0.000000
7,1475649,1025,0.400000
8,491883,612,-0.666667
9,281076,288,-0.428571


## Download tweets for a specific hashtag

In [ ]:
Moonbeam_df = tweet_hashtag_scrapper('(#Moonbeam) since:2022-09-20 until:2022-12-20', 200)
Moonbeam_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 21:41:04+00:00,sertekinler,17,2,Be sure to check out @Beamswapio 12 days of #D...,"[DeFi, Moonbeam]"
1,2022-12-19 21:38:17+00:00,sertekinler,17,2,Be sure to check out @Beamswapio 12 days of #D...,"[DeFi, Moonbeam]"
2,2022-12-19 21:00:57+00:00,Moonbeam_Daily,9466,10,#Moonbeam $GLMR &amp; #Moonriver $MOVR Social ...,"[Moonbeam, Moonriver]"
3,2022-12-19 20:57:35+00:00,Verasity73561,24,1,Be sure to check out @Beamswapio 12 days of #D...,"[DeFi, Moonbeam]"
4,2022-12-19 20:54:58+00:00,Verasity73561,24,1,Be sure to check out @Beamswapio 12 days of #D...,"[DeFi, Moonbeam]"


In [ ]:
Acala_df = tweet_hashtag_scrapper('(#AcalaNetwork) since:2022-09-20 until:2022-12-20', 200)
Acala_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-17 13:55:55+00:00,javonnnm,23047,2,$ACA #ACA #AcalaNetwork's\n\ncurrent target is...,"[ACA, AcalaNetwork]"
1,2022-12-14 11:18:54+00:00,CoinFyi,2287,6,Daily Gainers\n\n#SymbiosisFinance $SIS - 70.9...,"[SymbiosisFinance, Saddle, AcalaNetwork, Fetch..."
2,2022-12-03 10:58:34+00:00,PolkaRomania,200,5,Rezumat #PolkaDot \n🟢#AstarNetwork deschid can...,"[PolkaDot, AstarNetwork, PhalaNetwork, AstarNe..."
3,2022-12-03 03:00:27+00:00,BonelBot,93,0,The real price of Acala is $2.6 and currently ...,[AcalaNetwork]
4,2022-12-02 15:45:39+00:00,javonnnm,23047,2,$ACA #ACA #AcalaNetwork's\n\nprices on the 1d ...,"[ACA, AcalaNetwork]"


In [ ]:
Astar_df = tweet_hashtag_scrapper('(#AstarNetwork) since:2022-09-20 until:2022-12-20', 200)
Astar_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 15:46:32+00:00,neurolauncher,5648,201,If we think of #Polkadot as a decentralized un...,"[Polkadot, AstarNetwork, ASTR]"
1,2022-12-19 12:55:21+00:00,P85Crypto,3739,4,🎅🎅 #bitcoin \n\n#crypto #bitcoin #bitcoinart #...,"[bitcoin, crypto, bitcoin, bitcoinart, blockch..."
2,2022-12-19 12:42:44+00:00,CryptoPEN7,453,3,AstarFarmで地味にステーキングしてます。お野菜たちがすくすくと育っていて良い感じです...,"[ASTR, AstarNetwork]"
3,2022-12-19 05:28:21+00:00,tsarah0822,512,39,AMAでの @Sota_Web3 氏の言葉\n「Astar はオープンソースなので技術はコピ...,[AstarNetwork]
4,2022-12-18 13:11:43+00:00,uiuippa,423,0,#AstarNetwork 渡辺創太氏\n#平井拓也初代デジタル大臣\n#平将明自民党Web...,"[AstarNetwork, 平井拓也初代デジタル大臣, 平将明自民党Web3プロジェクトチ..."


In [ ]:
Litentry_df = tweet_hashtag_scrapper('(#Litentry) since:2022-09-20 until:2022-12-20', 200)
Litentry_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-15 09:11:46+00:00,CentralCharts,8582,2,$LIT $LITUSDT #LITENTRY - Journalier https://t...,[LITENTRY]
1,2022-12-15 09:11:46+00:00,LaunayVinc,4596,3,$LIT $LITUSDT #LITENTRY - Journalier https://t...,[LITENTRY]
2,2022-12-15 01:09:47+00:00,SnylexSwing,628,7,$LIT / $USDT\n\nExchange: @Binance\n\nProyecto...,"[LIT, Litentry, LITUSDT, Bitcoin, Binance, Cri..."
3,2022-12-14 19:46:14+00:00,LitentryRu,147,1,"Litentry хотел бы представить 5⃣x коллаторов, ...","[Staking, Collators, Litentry, Parachain]"
4,2022-12-14 09:17:52+00:00,litentry,70100,27,"Now that we've come to an end, feel free to sh...","[Collators, LIT, Litentry, Parachain, Staking]"


In [ ]:
#DarwiniaNetwork
Darwinia_df = tweet_hashtag_scrapper('(#DarwiniaNetwork) since:2022-09-20 until:2022-12-20', 200)
Darwinia_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 14:26:49+00:00,Warmanjm,739,1,This fun give away.\n\n#DarwiniaNetwork https:...,[DarwiniaNetwork]
1,2022-12-15 15:10:42+00:00,Warmanjm,739,4,Exciting times. Now the team can focus on mer...,[DarwiniaNetwork]
2,2022-12-13 03:25:39+00:00,Warmanjm,739,3,Good to see the merge of Darwinia's solochain ...,[DarwiniaNetwork]
3,2022-12-06 03:30:07+00:00,Warmanjm,739,6,Another exciting AMA from DarwiniaAMAs. Looki...,[DarwiniaNetwork]
4,2022-12-05 21:05:45+00:00,DarwiniaNetwork,70012,25,Catch Megan @thetokenblonde_ from #DarwiniaNet...,[DarwiniaNetwork]


In [ ]:
BSY_df = tweet_hashtag_scrapper('(@Bestay_bsy) since:2022-09-20 until:2022-12-20', 200)
BSY_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags


In [ ]:
Diora_df = tweet_hashtag_scrapper('(#DioraNetwork) since:2022-09-20 until:2022-12-20', 200)
Diora_df.head(5)

,Date,User,FollowersCount,LikeCount,Content,Hashtags


In [ ]:
ShiftNrg_df = tweet_hashtag_scrapper('(#SHIFTHUB) since:2022-09-20 until:2022-12-20', 200)
ShiftNrg_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags


In [ ]:
Acurast_df = tweet_hashtag_scrapper('(#acurastnetwork) since:2022-09-20 until:2022-12-20', 200)
Acurast_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags


In [ ]:
Phala_df = tweet_hashtag_scrapper('(#Phalanetwork) since:2022-09-20 until:2022-12-20', 200)
Phala_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 23:54:41+00:00,RafaelCabilla,241,1,@PhalaNetwork se puso en marcha en @Filecoin\n...,"[PhalaNetwork, PhatContract]"
1,2022-12-19 23:29:19+00:00,YZER001,113,3,Number 2\n$PHA #PhalaNetwork #PhalaWorld https...,"[PhalaNetwork, PhalaWorld]"
2,2022-12-19 23:24:14+00:00,YZER001,113,3,Never Buy HIGH and Sell LOW. \n\nAs long you s...,"[PhalaNetwork, PhalaWorld]"
3,2022-12-19 23:11:52+00:00,YZER001,113,1,@yousuf_gorgin @Dennis23124955 @PhalaNetwork W...,[PhalaNetwork]
4,2022-12-19 23:03:02+00:00,YZER001,113,14,Will you vote YES for this proposal?\n$PHA #Ph...,"[PhalaNetwork, PhalaWorld, Gamerhash]"


In [ ]:
KILT_df = tweet_hashtag_scrapper('(#Kilt) since:2022-09-20 until:2022-12-20', 200)
KILT_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 21:46:33+00:00,SubZeroCoin,148,0,@nhtctnk #kilt @Kiltprotocol,[kilt]
1,2022-12-19 19:48:49+00:00,UniqkidsC,6,0,Like and share this pure awesomeness! \r\r #Ta...,"[Tartan, Tie, Textile, Plaid, Kilt, Red, Patte..."
2,2022-12-19 14:00:36+00:00,uk_expo_kilts,37,0,Black Leather Utility Kilt 3 Straps\nUKH-528-1...,"[Black, Leather, Utility, Kilt, 3Straps]"
3,2022-12-19 13:59:23+00:00,MeijerChaos,806,1,"Another day! It's December 19, 2022 at 02:59PM...",[Kilt]
4,2022-12-19 11:18:26+00:00,SAVE_NEVER,165,0,@StreamerGro I want to never (pun Intended) ta...,"[hoodie, Kilt, streamer, lonely𓁿, catgirl]"


In [ ]:
Efinity_df = tweet_hashtag_scrapper('(#Efinity) since:2022-09-20 until:2022-12-20', 200)
Efinity_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 15:32:55+00:00,1Bliver,23,0,Please Don't Ignore the Animal Deaths. Help Us...,"[EOS, EXMO, Ecoin, Efforce, Efinity]"
1,2022-12-19 13:31:42+00:00,1Bliver,23,0,Upright a tree to new forests for your childre...,"[EOS, EXMO, Ecoin, Efforce, Efinity]"
2,2022-12-19 13:15:38+00:00,balayadav888,225,2,"@meltelbrot Thanks , received 🤝\n#Enjin #Efini...","[Enjin, Efinity, Enj, Efi]"
3,2022-12-18 19:19:30+00:00,1Bliver,23,0,Do not be a spectator to the pollution of the ...,"[EOS, EXMO, Ecoin, Efforce, Efinity]"
4,2022-12-18 01:10:10+00:00,bohorre,12,0,@ScottieTash @enjin 0xF202911c1691B3e925893CBe...,"[BadBearsFamily, efinity, EnjinNFT]"


In [ ]:
Centrifuge_df = tweet_hashtag_scrapper('(@centrifuge) since:2022-09-20 until:2022-12-20', 200)
Centrifuge_df.head()

,Date,User,FollowersCount,LikeCount,Content,Hashtags
0,2022-12-19 22:18:39+00:00,CruzerDefi,369,3,Kickoff the Holiday Season Week with a warm me...,None
1,2022-12-19 21:29:54+00:00,eB_crypto1,74,1,@PolkadotInsider @Polkadot @AstarNetwork @Phal...,None
2,2022-12-19 19:30:00+00:00,PolkadotInsider,21508,114,👀 Let's see the top Auction Winners on @Polkad...,None
3,2022-12-19 17:37:18+00:00,OctavioNotPunk,209,14,While cooking some interesting stuff with @Val...,None
4,2022-12-19 15:36:30+00:00,TheSTOShow,2937,9,Security Token Show 170 is LIVE🗣️\n\nThe Marke...,[STSCOTW]


## Apply the sentiment analysis function to the hashtag column

#### Moonbeam Project

In [ ]:
Moonbeam_df['sentiment_analysis'] = Moonbeam_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = Moonbeam_df["sentiment_analysis"].value_counts()
percentages = counts / len(Moonbeam_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     51.24%
Positive    48.76%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the max value and save it to a dataframe
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_moonbeam = pd.DataFrame({"project_name": ["Moonbeam"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_moonbeam)


  project_name    label  max_value
0     Moonbeam  Neutral  51.243781


#### Acala Project

In [ ]:
Acala_df['sentiment_analysis'] = Acala_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = Acala_df["sentiment_analysis"].value_counts()
percentages = counts / len(Acala_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     85.00%
Positive    10.00%
Negative     5.00%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Acala = pd.DataFrame({"project_name": ["Acala"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Acala)

  project_name    label  max_value
0        Acala  Neutral       85.0


#### Astar Project

In [ ]:
Astar_df['sentiment_analysis'] = Astar_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = Astar_df["sentiment_analysis"].value_counts()
percentages = counts / len(Astar_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     78.11%
Positive    21.89%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Astar = pd.DataFrame({"project_name": ["Astar"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Astar)

  project_name    label  max_value
0        Astar  Neutral  78.109453


#### Litentry Project

In [ ]:
Litentry_df['sentiment_analysis'] = Litentry_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = Litentry_df["sentiment_analysis"].value_counts()
percentages = counts / len(Litentry_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     78.61%
Positive    18.41%
Negative     2.99%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Litentry = pd.DataFrame({"project_name": ["Litentry"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Litentry)

  project_name    label  max_value
0     Litentry  Neutral  78.606965


#### Darwinia Project

In [ ]:
Darwinia_df['sentiment_analysis'] = Darwinia_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = Darwinia_df["sentiment_analysis"].value_counts()
percentages = counts / len(Darwinia_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Positive    61.73%
Neutral     38.27%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Darwinia = pd.DataFrame({"project_name": ["Darwinia"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Darwinia)

  project_name     label  max_value
0     Darwinia  Positive  61.728395


#### BSY Project

In [ ]:
BSY_df['sentiment_analysis'] = BSY_df['Content'].apply(twitter_sentiment)
# Calculate the overall percentage

# Calculate the percentage of each value in the "sentiment" column
counts = BSY_df["sentiment_analysis"].value_counts()
percentages = counts / len(BSY_df) * 100

# print(percentages)
print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Series([], Name: sentiment_analysis, dtype: float64)


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_BSY = pd.DataFrame({"project_name": ["BSY"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_BSY)

#### Diora Project

In [ ]:
Diora_df['sentiment_analysis'] = Diora_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = Diora_df["sentiment_analysis"].value_counts()
percentages = counts / len(Diora_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Series([], Name: sentiment_analysis, dtype: float64)


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Diora = pd.DataFrame({"project_name": ["Diora"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Diora)

#### ShiftNrg Project

In [ ]:
ShiftNrg_df['sentiment_analysis'] = ShiftNrg_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = ShiftNrg_df["sentiment_analysis"].value_counts()
percentages = counts / len(ShiftNrg_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Series([], Name: sentiment_analysis, dtype: float64)


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_ShiftNrg = pd.DataFrame({"project_name": ["ShiftNrg"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_ShiftNrg)

#### Acurast Project

In [ ]:
Acurast_df['sentiment_analysis'] = Acurast_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = Acurast_df["sentiment_analysis"].value_counts()
percentages = counts / len(Acurast_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Series([], Name: sentiment_analysis, dtype: float64)


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Acurast = pd.DataFrame({"project_name": ["Acurast"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Acurast)

#### Phala Project

In [ ]:
Phala_df['sentiment_analysis'] = Phala_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = Phala_df["sentiment_analysis"].value_counts()
percentages = counts / len(Phala_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))


Neutral     65.67%
Positive    31.84%
Negative     2.49%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Phala = pd.DataFrame({"project_name": ["Phala"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Phala)

  project_name    label  max_value
0        Phala  Neutral  65.671642


#### KILT Project

In [ ]:
KILT_df['sentiment_analysis'] = KILT_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = KILT_df["sentiment_analysis"].value_counts()
percentages = counts / len(KILT_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     60.20%
Positive    38.31%
Negative     1.49%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_KILT = pd.DataFrame({"project_name": ["KILT"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_KILT)

  project_name    label  max_value
0         KILT  Neutral  60.199005


#### Efinity Project

In [ ]:
Efinity_df['sentiment_analysis'] = Efinity_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = Efinity_df["sentiment_analysis"].value_counts()
percentages = counts / len(Efinity_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     57.71%
Positive    38.81%
Negative     3.48%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Efinity = pd.DataFrame({"project_name": ["Efinity"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Efinity)

  project_name    label  max_value
0      Efinity  Neutral  57.711443


#### Centrifuge Project

In [ ]:
Centrifuge_df['sentiment_analysis'] = Centrifuge_df['Content'].apply(twitter_sentiment)

# Calculate the percentage of each value in the "sentiment" column
counts = Centrifuge_df["sentiment_analysis"].value_counts()
percentages = counts / len(Centrifuge_df) * 100

print(percentages.apply(lambda x: "{:.2f}%".format(round(x, 2))))

Neutral     50.25%
Positive    45.27%
Negative     4.48%
Name: sentiment_analysis, dtype: object


In [ ]:
# Get the label of the maximum value
label = percentages.idxmax()

# Get the maximum value
max_value = percentages.max()

# Create a dataframe with the label, max_value, and project name
sentiment_result_Centrifuge = pd.DataFrame({"project_name": ["Centrifuge"], "label": [label], "max_value": [max_value]})

# Display the dataframe
print(sentiment_result_Centrifuge)

  project_name    label  max_value
0   Centrifuge  Neutral  50.248756


#### Concaterate all the dataframes

In [ ]:
# List of dataframes to concatenate
df_list = [sentiment_result_moonbeam, sentiment_result_Acala, sentiment_result_Astar, sentiment_result_Litentry, 
           sentiment_result_Darwinia, sentiment_result_Phala, sentiment_result_KILT, 
           sentiment_result_Efinity, sentiment_result_Centrifuge]  # 没有数据  sentiment_result_BSY,sentiment_result_Diora, sentiment_result_ShiftNrg, sentiment_result_Acurast

# Initialize an empty dataframe
hastagSentiment = pd.DataFrame()

# Concatenate the dataframes one by one
for i, df_ in enumerate(df_list):
    hastagSentiment = pd.concat([hastagSentiment, df_], ignore_index=True)


hastagSentiment

,project_name,label,max_value
0,Moonbeam,Neutral,51.243781
1,Acala,Neutral,85.000000
2,Astar,Neutral,78.109453
3,Litentry,Neutral,78.606965
4,Darwinia,Positive,61.728395
5,Phala,Neutral,65.671642
6,KILT,Neutral,60.199005
7,Efinity,Neutral,57.711443
8,Centrifuge,Neutral,50.248756
